In [2]:
import mysql.connector
import pandas as pd
import numpy as np
import os
from dotenv import load_dotenv
load_dotenv(verbose=True)

# Database connection parameters
DB_CONFIG = {
    "host": os.getenv("DB_HOST"),
    "user": os.getenv("DB_USER"),
    "password": os.getenv("DB_PASSWORD"),
    "database": os.getenv("DB_NAME"),
}

print('Database URL: ' + os.environ.get('DB_HOST'))

Database URL: timetables.mysql.database.azure.com


In [3]:
# Fetch data from database and return as a DataFrame
def fetch_data(table_name, column_names='*', condition='1'):
    try:
        # Connect
        conn = mysql.connector.connect(**DB_CONFIG)
        cursor = conn.cursor()
        # Fetch
        query = f"SELECT {column_names} FROM {table_name} WHERE {condition}"
        cursor.execute(query)
        # Fetch column names
        columns = [col[0] for col in cursor.description]
        # Fetch data
        data = cursor.fetchall()
        df = pd.DataFrame(data, columns=columns)
        return df
    except mysql.connector.Error as err:
        print(f"Error: {err}")
        return None
    finally:
        if 'conn' in locals() and conn.is_connected():
            cursor.close()
            conn.close()

In [4]:
# Query the running balance of the asset classes
table_name = 'assetclassbalances'
column_names = 'TransactionDate, Portfolio, Cash_Sweep, US_Treasuries, CDs, CommercialPapers, MoneyMarket, MutualFund, US_Agencies'
condition = 'Title '
assets = fetch_data(table_name, column_names)

if assets is not None:
    print(assets.head())

  TransactionDate     Portfolio    Cash_Sweep US_Treasuries         CDs  \
0      2025-01-21  654628445.37  151070374.54  163716733.42  2600000.00   
1      2025-01-22  492520445.37   37442374.54  163716733.42  2600000.00   
2      2025-01-23  492520445.37   47442374.54  163716733.42  2600000.00   
3      2025-01-24  470088445.37   33510374.54  163716733.42  2600000.00   
4      2025-01-25  470088445.37   33510374.54  163716733.42  2600000.00   

  CommercialPapers  MoneyMarket   MutualFund   US_Agencies  
0     170669204.47  20065580.67  18975581.76  127530970.51  
1     156169204.47  20065580.67  18975581.76   93550970.51  
2     146169204.47  20065580.67  18975581.76   93550970.51  
3     137669204.47  20065580.67  18975581.76   93550970.51  
4     137669204.47  20065580.67  18975581.76   93550970.51  


In [5]:
# Query the asset class max. percentages
table_name = 'AssetClass'
column_names = 'Title AS "AssetClass", PercentMax'
condition = 'AssetClassParentID = 0 && Title != "Not Assigned"'
max_percent = fetch_data(table_name, column_names, condition)
if max_percent is not None:
    # Insert an asset class key for column mapping
    asset_class_mapping = {
        'Cash/Sweep': 'Cash_Sweep',
        'US Treasuries': 'US_Treasuries',
        'Certificate of Deposit': 'CDs',
        'Commercial Paper': 'CommercialPapers',
        'Money Market': 'MoneyMarket',
        'Mutual Fund': 'MutualFund',
        'US Agencies': 'US_Agencies'
    }
    max_percent['AssetClassKey'] = max_percent['AssetClass'].map(asset_class_mapping)

    # Convert PercentMax to float
max_percent['PercentMax'] = max_percent['PercentMax'].astype(float)

print(max_percent)
# max_percent.info()

               AssetClass  PercentMax     AssetClassKey
0              Cash/Sweep        1.00        Cash_Sweep
1  Certificate of Deposit        0.50               CDs
2        Commercial Paper        0.50  CommercialPapers
3             US Agencies        0.25       US_Agencies
4            Money Market        0.50       MoneyMarket
5             Mutual Fund        0.50        MutualFund
6           US Treasuries        0.50     US_Treasuries


In [33]:
# Get the available cash min and max values
min_cash = assets['Cash_Sweep'].min()
max_cash = assets['Cash_Sweep'].max()
print(f"Min:{min_cash:,}" + '  ' f"Max:{max_cash:,}")

# Create a df with values between min_cash and max_cash (inclusive) named 'Cash'.
def prorate_cash_values(num_values, min_cash, max_cash):
    prorated_values = pd.Series(np.linspace(min_cash, max_cash, num_values))
    prorated_df = pd.DataFrame({'Cash': prorated_values})
    prorated_df['Cash'] = prorated_df['Cash'].astype(int)
    return prorated_df

# Prorate the cash interval values
min_investment = 100000
num_values = int((max_cash - min_cash) / min_investment)
print(num_values)

prorated_df = prorate_cash_values(num_values, min_cash, max_cash)
prorated_df


Min:28,568,374.54  Max:387,489,374.54
3589


,Cash
0,28568374
1,28668408
2,28768441
3,28868475
4,28968509
...,...
3584,387089239
3585,387189273
3586,387289307
3587,387389340


In [26]:
# Check the increment delta by subtracting the second cash value from the first
increment = prorated_df['Cash'].iloc[1] - prorated_df['Cash'].iloc[0]
f"For {num_values:,} prorated values the increment is: ${increment:,}"

'For 3,589 prorated values the increment is: $100,034'

In [45]:

algo_df = assets[['TransactionDate', 'Cash_Sweep']].sort_values(by='TransactionDate', ascending=True).rename(columns={'TransactionDate': 'Date', 'Cash_Sweep': 'Cash'})
algo_df

,Date,Cash
0,2025-01-21,151070374.54
1,2025-01-22,37442374.54
2,2025-01-23,47442374.54
3,2025-01-24,33510374.54
4,2025-01-25,33510374.54
...,...,...
156,2025-06-26,258505374.54
157,2025-06-27,257205374.54
158,2025-06-28,257205374.54
159,2025-06-29,257205374.54


In [ ]:



# For each cash amount in the prorated_df create a column in a results DataFrame
# to store the bool check if the cash amount exceeds the cash_sweep balance, and the bool check if the cash amount exceeds the max percent
# include the date and cash_sweep balance in results.

# results = pd.DataFrame()

# for index, row in assets.iterrows():
#     date = row['TransactionDate']
#     cash_sweep_balance = float(row['Cash_Sweep'])
#     max_percent_balance = cash_sweep_balance * max_percent.loc[max_percent['AssetClassKey'] == 'Cash_Sweep', 'PercentMax'].values[0]

#     results[date] = prorated_df['Cash'].apply(lambda x: (x > cash_sweep_balance, x > max_percent_balance))

# results = results.T
# results.columns = pd.MultiIndex.from_product([prorated_df['Cash'], ['Exceeds_Cash_Sweep', 'Exceeds_Max_Percent']])
# results.index.name = 'TransactionDate'
# results.reset_index(inplace=True)

# results



In [39]:
results.columns

RangeIndex(start=0, stop=3589, step=1)